In [1]:
import pandas as pd
from datetime import timedelta
import pandas as pd
import numpy as np
from rpy2.robjects import pandas2ri
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from scipy.stats import f
from sklearn import metrics
from sklearn.model_selection import KFold
import rpy2
import numpy as np

import os
os.chdir(os.getcwd()+"/../..")
main_folder=os.getcwd()

pandas2ri.activate()
mgcv = importr('mgcv')
VIF=importr('VIF')

In [2]:
all_data_encounter=pd.read_csv(main_folder+"/Supplement data/idealist_encounters.csv")

/home/mmadkour/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (84) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df=pd.read_csv(main_folder+"/Dev_Cohort/development_cohort_"+"cardiovasc_comp_comb"+".csv")

In [4]:
my_list=list(df["encounter_deiden_id"])

In [5]:
all_data_encounter=all_data_encounter[all_data_encounter["encounter_deiden_id"].isin(my_list)]

In [6]:
all_data_encounter["death_date"] = pd.to_datetime(all_data_encounter["death_date"])
all_data_encounter["admit_datetime"] = pd.to_datetime(all_data_encounter["admit_datetime"])
all_data_encounter['admission_to_death_time']=all_data_encounter[["death_date","admit_datetime"]].apply(lambda x:(x[0]-x[1])/timedelta(days=365.2425) if (x[0]==x[0] and x[1]==x[1]) else None,axis=1)

In [ ]:


df2=all_data_encounter[~(all_data_encounter["ssdi_death_date"].isnull())]
df3=df2[["death_date","ssdi_death_date"]]

In [ ]:
try:
    df3["death_date"] = pd.to_datetime(df3["death_date"])
except:
    print 1

In [ ]:
df3.loc[30614,"death_date"]=pd.to_datetime(df3.loc[30614,"death_date"])

In [ ]:
for i in df3.index:
    try:
        df3.loc[i,"death_date"]=pd.to_datetime(df3.loc[i,"death_date"])
        print type(df3.loc[i,"death_date"])
    except: 
        print type(df3.loc[i,"death_date"])

In [ ]:
df3.size

In [ ]:
for i in df3.index:
    print type(df3.loc[i,"death_date"])
    if (str(type(df3.loc[i,"death_date"]))=="<type 'str'>"):
        print 1
        df3.drop(i)

        

In [ ]:
df3

In [ ]:
#all_data_encounter["ssdi_death_date"]
i=i+20
df3.iloc[i:i+20,:]

In [ ]:
df3["death_date"]=datetime.Timestamp(df3["death_date"])

In [ ]:
import datetime

In [ ]:
type(df2.loc[206785,"ssdi_death_date"])

In [ ]:
(type(df3["death_date"]))
for index, row in df3.iterrows():
    if (str(type(row["death_date"]))=="<class 'pandas._libs.tslib.NaTType'>"):
        df3["death_date"]=df3["ssdi_death_date"]
        print 1
    


In [ ]:
nat_as_integer = np.datetime64('NAT').view('i8')

def isnat(your_datetime):
    dtype_string = "Timestamp"
    if 'datetime64' in dtype_string or 'timedelta64' in dtype_string:
        return your_datetime.view('i8') == nat_as_integer
    return False  # it can't be a NaT if it's not a dateime

In [ ]:
df2[["death_date","ssdi_death_date"]]

In [ ]:
all_data_encounter=all_data_encounter[["encounter_deiden_id",'admission_to_death_time']]

In [ ]:
df=all_data_encounter

In [ ]:
df

In [ ]:
df["mort_status_30d"]=(df["admission_to_death_time"]<=.085).astype(int)
df["mort_status_90d"]=(df["admission_to_death_time"]<=.25).astype(int)
df["mort_status_6m"]=(df["admission_to_death_time"]<=.5).astype(int)
df["mort_status_1y"]=(df["admission_to_death_time"]<=1).astype(int)
df["mort_status_2y"]=(df["admission_to_death_time"]<=2).astype(int)
df.to_csv(main_folder+"/Mortality/mortality_data.csv")
df

In [ ]:
statsf = importr('stats',robject_translations = {'format_perc': 'format_dot_perc'})
risknames = ['aki_3','aki_7', 'aki_all','MV_duration','cardiovasc_comp_comb', 'ICU_duration', 'neuro_delirium_comb', 'post_op_mech_proc_comb', 'sepsis_comb', 'venos_throm_comb']

In [ ]:

for riskname in risknames:
    print ("Model building for", riskname)
    #os.chdir(main_folder+"/Models/"+riskname+" .Rdata")
    ro.globalenv['riskname']=riskname
    ro.globalenv['main_folder']=main_folder
    ro.r['load'](main_folder+"/Models/"+riskname+" .Rdata")
    best_model_cvcomp=ro.globalenv['best_model']
    #os.chdir(main_folder)
    #ro.r('load(best_model,file=paste(location,".Rdata"))')
    #testdata=pd.read_csv(main_folder+"/Val_Cohort/test_cohort_"+riskname+".csv")
    devdata=pd.read_csv(main_folder+"/Dev_Cohort/development_cohort_"+riskname+".csv")
    ro.globalenv['r_df_cvcomp'] = pandas2ri.py2ri(devdata)
    r_df_cvcomp=ro.r('r_df_cvcomp')
    pred_score_cvcomp=statsf.predict(best_model_cvcomp,r_df_cvcomp, type="response")
    devdata.rename(columns={'outcome':riskname}, inplace=True)
    devdata[riskname]=pred_score_cvcomp
    devdata=devdata[["encounter_deiden_id",riskname]]
    df=pd.merge(df,devdata,on="encounter_deiden_id",how="outer")
df.to_csv(main_folder+"/Mortality/idealist_mortality.csv")

In [ ]:
df.to_csv("idealist_mortality.csv")

In [ ]:
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.model_selection  import GridSearchCV 
from sklearn.externals import joblib

In [ ]:
df

In [ ]:
df3=df[pd.notnull(df[['aki_all','aki_3','aki_7']]).all(axis=1)]

In [ ]:
df3.to_csv(main_folder+"/Mortality/idealist_mortality2.csv")

In [ ]:
df=pd.read_csv(main_folder+"/Mortality/idealist_mortality.csv")

In [ ]:
df.shape

In [ ]:
def run_rf_model(data, colnames): 

    Kfold_value=5 #number of folds
  
    data=data[pd.notnull(data).all(axis=1)]
    
    features =data[data.columns.difference([colnames])]
   
    print("Running Random_Forest_Classifire")
    clf=RFC(n_jobs=2)
    
    param_grid={"n_estimators"      : [300], #tuning the random forest with these parameters
                "criterion"         : ["gini"],
                'max_features': [ 'sqrt', 5],
                "max_depth"         : [None ]}
    grid_search = GridSearchCV(clf, param_grid, scoring='accuracy',n_jobs=2, cv=Kfold_value) #acu metric used to find the best model
    grid_search.fit(features, data[colnames])
    model=grid_search.best_estimator_
    acu=grid_search.best_score_
  
    return model,acu

In [ ]:
models=["mort_status_30d","mort_status_90d","mort_status_6m","mort_status_1y","mort_status_2y"]
feature_names= ['cardiovasc_comp_comb','ICU_duration','neuro_delirium_comb','post_op_mech_proc_comb','sepsis_comb',
                'venos_throm_comb','aki_all','MV_duration','aki_3','aki_7']
for model in models:
    mdl,acu=run_rf_model(df[feature_names+[model]],model)
    print("acu for "+model+' is: '+str(acu))
    joblib.dump(mdl, main_folder+'/M_Models/'+model+'_RandomForestClassifier.pkl')